# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bata,1.8639,9.7658,299.83,86,84,2.66,GQ,1706499433
1,1,ulubey,40.8721,37.7588,274.22,97,100,1.33,TR,1706499725
2,2,alice springs,-23.7000,133.8833,313.83,11,23,4.12,AU,1706499725
3,3,ilulissat,69.2167,-51.1000,250.16,49,75,5.14,GL,1706499626
4,4,lihue,21.9789,-159.3672,300.02,77,40,8.75,US,1706499726


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha=.75
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bata,1.8639,9.7658,299.83,86,84,2.66,GQ,1706499433
1,1,ulubey,40.8721,37.7588,274.22,97,100,1.33,TR,1706499725
2,2,alice springs,-23.7000,133.8833,313.83,11,23,4.12,AU,1706499725
3,3,ilulissat,69.2167,-51.1000,250.16,49,75,5.14,GL,1706499626
4,4,lihue,21.9789,-159.3672,300.02,77,40,8.75,US,1706499726


In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_criteria = city_data_df.loc[(city_data_df['Max Temp'] >= 300) &   
                                  (city_data_df['Wind Speed'] <= 20) & 
                                    (city_data_df['Cloudiness'] <= 30)]
# Drop results with null values
weather_criteria = weather_criteria.dropna()

# Display sample data
weather_criteria

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,alice springs,-23.7000,133.8833,313.83,11,23,4.12,AU,1706499725
21,21,carnarvon,-24.8667,113.6333,300.67,56,0,7.26,AU,1706499242
41,41,west island,-12.1568,96.8225,302.14,65,20,4.63,CC,1706499185
131,131,kudahuvadhoo,2.6708,72.8944,300.78,78,30,6.67,MV,1706499749
159,159,santana,-0.0583,-51.1817,300.25,89,0,2.06,BR,1706499753
204,204,neiafu,-18.6500,-173.9833,302.53,70,20,3.60,TO,1706499759
208,208,georgetown,5.4112,100.3354,305.90,60,20,6.17,MY,1706499545
227,227,kratie,12.4881,106.0188,305.52,36,27,4.62,KH,1706499761
278,278,kavaratti,10.5669,72.6420,300.61,69,29,4.13,IN,1706499224
293,293,mildura,-34.2000,142.1500,308.75,25,0,2.72,AU,1706499769


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = weather_criteria[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,alice springs,AU,-23.7000,133.8833,11,
21,carnarvon,AU,-24.8667,113.6333,56,
41,west island,CC,-12.1568,96.8225,65,
131,kudahuvadhoo,MV,2.6708,72.8944,78,
159,santana,BR,-0.0583,-51.1817,89,
204,neiafu,TO,-18.6500,-173.9833,70,
208,georgetown,MY,5.4112,100.3354,60,
227,kratie,KH,12.4881,106.0188,36,
278,kavaratti,IN,10.5669,72.6420,69,
293,mildura,AU,-34.2000,142.1500,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"category":"accommodation.hotel",
          "apiKey":geoapify_key
}

In [8]:
# Print a message to follow up the hotel search
print("Starting hotel search")

Starting hotel search


In [9]:
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},10000"
    params["bias"] = f"circle:{lng},{lat},1000"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

alice springs - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
kudahuvadhoo - nearest hotel: No hotel found
santana - nearest hotel: No hotel found
neiafu - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
kratie - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
mildura - nearest hotel: No hotel found
male - nearest hotel: No hotel found
sao tome - nearest hotel: No hotel found
ca mau - nearest hotel: No hotel found
san andres - nearest hotel: No hotel found
malabar - nearest hotel: No hotel found
champerico - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,alice springs,AU,-23.7000,133.8833,11,No hotel found
21,carnarvon,AU,-24.8667,113.6333,56,No hotel found
41,west island,CC,-12.1568,96.8225,65,No hotel found
131,kudahuvadhoo,MV,2.6708,72.8944,78,No hotel found
159,santana,BR,-0.0583,-51.1817,89,No hotel found
204,neiafu,TO,-18.6500,-173.9833,70,No hotel found
208,georgetown,MY,5.4112,100.3354,60,No hotel found
227,kratie,KH,12.4881,106.0188,36,No hotel found
278,kavaratti,IN,10.5669,72.6420,69,No hotel found
293,mildura,AU,-34.2000,142.1500,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["Hotel Name","Country"])

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)